# Calculating Density-controlled Vowel Space Area

This notebook applies the methodology demonstrated in [Story & Bunton (2017)](https://doi.org/10.1121/1.4983342) for calculating density-controlled vowel space area, ideally for measuring vowel space area across task types or mediums. Formant measures are obtained at 5ms intervals of the vowel duration and then normalized. A (200 x 200) grid is generated and for each point on the grid a density value is obtained, equal to the number of formant data points that fall within a 0.05 radius field-of-view around the grid point. A heatmap is created with scaled density values, showing the relative concentrations of vowel productions. A convex hull measure is taken at several density cut-offs.

In [1]:
import pandas as pd
import numpy as np

In [2]:
spanish = pd.read_csv("data/spanish_vowels.csv")
english = pd.read_csv("data/english_vowels.csv")

## Define function to calculate density-controlled vowel space area

In [3]:
def vsd(df, vowel_column):
    import pandas as pd
    import numpy as np
    from scipy.spatial import ConvexHull
    from scipy.spatial import distance
    
    # get lists of participants in dataset
    parts = list(df['Participant'].unique())
    vowels_list = list(df[vowel_column].unique())
        
    # define empty list to fill with participant values in form of dataframes
    areas_all = []
    
    # define necessary functions and global variables
    ################################################################################
    def rem_outliers(df, vowel_column):
        print("Removal of outliers:")
        print("Initial length: ", len(df))
    
        # establish 25% and 75% for each formant
        f1_qrts = df.groupby(['Participant', vowel_column])["F1"].describe()[['25%', '75%']]
        f2_qrts = df.groupby(['Participant', vowel_column])["F2"].describe()[['25%', '75%']]
    
        # find interquartile range for each formant
        f1_qrts['IQR'] = f1_qrts['75%'] - f1_qrts['25%']
        f2_qrts['IQR'] = f2_qrts['75%'] - f2_qrts['25%']
    
        # determine upper limit for each formant
        f1_qrts['upper'] = f1_qrts['75%'] + (1.5 * f1_qrts['IQR'])
        f2_qrts['upper'] = f2_qrts['75%'] + (1.5 * f2_qrts['IQR'])
    
        # determine lower limit for each formant
        f1_qrts['lower'] = f1_qrts['25%'] - (1.5 * f1_qrts['IQR'])
        f2_qrts['lower'] = f2_qrts['25%'] - (1.5 * f2_qrts['IQR'])
    
        # create smaller df with only limits for each formant
        f1_limits = f1_qrts[['upper','lower']]
        f2_limits = f2_qrts[['upper','lower']]
    
        # merge limits into original df
        df = df.merge(f1_limits, left_on = ["Participant", vowel_column], right_index = True)
        df = df.merge(f2_limits, left_on = ["Participant", vowel_column], right_index = True, suffixes = ("_f1", "_f2"))
    
        # drop rows with outlier formants
        df = df[(df["F1"] > df["lower_f1"]) & (df["F1"] < df["upper_f1"])]
        df = df[(df["F2"] > df["lower_f2"]) & (df["F2"] < df["upper_f2"])]

        print("Final length: ", len(df))
        return df
    
    ####################################################################################
    # define function to scale all formant measures
    def scale_formants(df):
        medians = df.groupby(["Participant", "is_stress"])[["F1", "F2"]].median()

        df = df.merge(medians, left_on = ["Participant", "is_stress"], right_index = True, suffixes = ("","_med"))
        df["F1_vsd"] = (df["F1"]-df["F1_med"])/df["F1_med"]
        df["F2_vsd"] = (df["F2"]-df["F2_med"])/df["F2_med"]
        
        print("Formant data scaled")
        return df 
    
    ####################################################################################
    
    # define variable `grid`
    xvalues = np.flip(np.arange(-1., 1.01, 0.01))
    yvalues = np.arange(-1., 1.01, 0.01)

    grid = [(round(x, 2), round(y,2)) for x in xvalues for y in yvalues]
      
    #####################################################################################
    
    # define function get_density
    def density(grid, df):
        density_dict = {}
        for c in grid:
            # define x and y coordinate
            x = c[0]
            y = c[1]
    
            # define max and min coordinates that form a square of length 0.1 around the coordinate
            x_max = x + 0.05
            x_min = x - 0.05
            y_max = y + 0.05
            y_min = y - 0.05
    
            # pull a subset of data that fall within the box
            opts = df[(df["F2_vsd"] <= x_max) &
               (df["F2_vsd"] >= x_min) &
               (df["F1_vsd"] <= y_max) &
               (df["F1_vsd"] >= y_min)].copy()
            opts_list = list(zip(opts["F2_vsd"], opts["F1_vsd"]))
    
             # define density for this point
            dens = 0
    
            for o in opts_list:
                # define x and y coordinate of each point in my data
                o_x = o[0]
                o_y = o[1]
        
                # calculate distance to grid point
                dist = distance.euclidean([x, y], [o_x, o_y])
        
                if dist <= 0.05:
                    dens += 1
        
            density_dict[c] = dens
    
        # convert to df
        density_df = pd.DataFrame(density_dict.items())
        density_df = density_df.rename(columns = {0: "coord", 1:"density"})
    
        # make grid into df
        grid_df = pd.DataFrame(grid, columns=["x", "y"])
        grid_df["coord"] = list(zip(grid_df["x"], grid_df["y"]))
    
        # join df and grid_df
        grid_df = grid_df.join(density_df.set_index('coord'), on="coord")
              
        return grid_df
    
    ##################################################################################
    # remove outliers by participant, by task, by vowel
    df = rem_outliers(df, vowel_column)
    
    # Lobanov normalization of formants, by participant by task
    df = scale_formants(df)
    
    for i in parts:
        print("\n",i)
        
        # get subset of data for speaker across modality
        df_stress = df[(df["Participant"]==i) & (df["is_stress"]== 1)].copy()
        df_unstress = df[(df["Participant"]==i) & (df["is_stress"]== 0)].copy()  
           
        # get density for each point across the modalities
        grid_stress_df = density(grid, df_stress)
        print("Densities calculated for stressed vowels")
        grid_unstress_df = density(grid, df_unstress)
        print("Densities calculated for unstressed vowels")
              
        # scale density measures
        grid_stress_df["density_norm"] = grid_stress_df["density"].apply(lambda x: x/grid_stress_df["density"].max())
        grid_unstress_df["density_norm"] = grid_unstress_df["density"].apply(lambda x: x/grid_unstress_df["density"].max())
        
        # pull out coordinates at various density cutoffs and get areas
        cutoffs = [0.10, 0.15, 0.20, 0.25, 0.3]

        areas_list = []

        for c in cutoffs:
            grid_stress_density = grid_stress_df[grid_stress_df["density_norm"] >= c].copy()
            grid_unstress_density = grid_unstress_df[grid_unstress_df["density_norm"] >= c].copy()
    
            points_tuples_stress = list(grid_stress_density["coord"])
            points_tuples_unstress = list(grid_unstress_density["coord"])
    
            points_stress = [list(k) for k in points_tuples_stress]
            points_unstress = [list(k) for k in points_tuples_unstress]
    
            hull_stress = ConvexHull(points_stress)
            hull_unstress = ConvexHull(points_unstress)
    
            area_stress = hull_stress.area
            area_unstress = hull_unstress.area

            areas = {"is_stress": [1, 0], "Area": [area_stress, area_unstress], "Cutoff": c}
            areas_list.append(areas)
            
        # to show progress while running, print areas at density cutoff of 0.25
        print("Stress, unstress areas at 0.25 cutoff are: ", areas_list[3]["Area"])
        
        # generate dataframe for this participant and reformat
        areas_df = pd.DataFrame(areas_list)
        areas_df_stress = areas_df.explode('is_stress')
        areas_df_stress = areas_df_stress.drop(["Area", "Cutoff"], axis = 1)
        areas_df_area = areas_df.explode('Area')
        areas_df_area = areas_df_area.drop("is_stress", axis = 1)
        areas_df = pd.concat([areas_df_stress, areas_df_area], axis = 1)
        areas_df["Participant"] = i

        # append dataframe to list of dataframes
        areas_all.append(areas_df)
 
    # return dataframe of areas
    areas_to_return = pd.concat(areas_all)
    
    return areas_to_return

In [4]:
spa_vsd = vsd(spanish, "Vowel")

Removal of outliers:
Initial length:  355115
Final length:  324111
Formant data scaled

 p111
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [2.5524433175361674, 2.7128532528829874]

 p113
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [2.6301294587911466, 2.1054029973381465]

 p114
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [2.666707564899484, 2.3660280216394174]

 p117
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [2.915201082725338, 3.0106180586550755]

 p118
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [2.858556038286394, 3.0197287305961265]

 p119
Densities calculated for stre

In [5]:
eng_vsd = vsd(english, "Vowel_bare")

Removal of outliers:
Initial length:  16524
Final length:  15453
Formant data scaled

 p111
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [3.0763469208390757, 2.1789235534106774]

 p113
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [3.7572730610212806, 2.7397367513194633]

 p114
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [3.3373550060606427, 2.1201743716229857]

 p117
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [3.0600573159516746, 3.4594645761921505]

 p118
Densities calculated for stressed vowels
Densities calculated for unstressed vowels
Stress, unstress areas at 0.25 cutoff are:  [3.570705075097101, 2.240494517139016]

 p120
Densities calculated for stres

Add in demographic information:

In [6]:
eng_dom = pd.DataFrame({"Participant": ['p111', 'p113', 'p114', 'p117','p118', 'p119', 'p120', 'p121', 'p122', 'p123', 'p124', 'p126'],
                       "Dom": ["L2", "biling", "biling", "L2", "biling", "biling", "biling", "L2", "biling", "L2", "biling", "L2"]})
eng_vsd = eng_vsd.merge(eng_dom, on = ["Participant"])
eng_vsd.head()

,is_stress,Area,Cutoff,Participant,Dom
0,1,4.017514,0.10,p111,L2
1,0,3.348619,0.10,p111,L2
2,1,3.508462,0.15,p111,L2
3,0,2.990106,0.15,p111,L2
4,1,3.429103,0.20,p111,L2


In [7]:
spa_dom = pd.DataFrame({"Participant": ['p111', 'p113', 'p114', 'p117','p118', 'p119', 'p120', 'p121', 'p122', 'p123', 'p124', 'p126', 's051', 's053', 's055', 's056', 's001', 's002'],
                       "Dom": ["L2", "biling", "biling", "L2", "biling", "biling", "biling", "L2", "biling", "L2", "biling", "L2", "mono", "mono", "mono", "mono", "mono", "mono"]})
spa_vsd = spa_vsd.merge(spa_dom, on=["Participant"])
spa_vsd.head()

,is_stress,Area,Cutoff,Participant,Dom
0,1,3.302681,0.10,p111,L2
1,0,3.539401,0.10,p111,L2
2,1,3.032504,0.15,p111,L2
3,0,3.023459,0.15,p111,L2
4,1,2.816748,0.20,p111,L2


Add in average duration of phones by speaker

In [8]:
spa_vowels = pd.read_csv("data/spanish_vowels.csv")
spadur = pd.DataFrame(spa_vowels.groupby(["Participant"])["avg_dur"].mean())
spadur = spadur.reset_index(drop = False)
spadur.head()

,Participant,avg_dur
0,p111,0.120470
1,p113,0.136585
2,p114,0.158571
3,p117,0.110605
4,p118,0.113432


In [9]:
spa_vsd = spa_vsd.merge(spadur, on=["Participant"])
spa_vsd.head()

,is_stress,Area,Cutoff,Participant,Dom,avg_dur
0,1,3.302681,0.10,p111,L2,0.12047
1,0,3.539401,0.10,p111,L2,0.12047
2,1,3.032504,0.15,p111,L2,0.12047
3,0,3.023459,0.15,p111,L2,0.12047
4,1,2.816748,0.20,p111,L2,0.12047


In [10]:
eng_vowels = pd.read_csv("data/english_vowels.csv")
engdur = pd.DataFrame(eng_vowels.groupby(["Participant"])["avg_dur"].mean())
engdur = engdur.reset_index(drop = False)
engdur.head()

,Participant,avg_dur
0,p111,0.121433
1,p113,0.148272
2,p114,0.176165
3,p117,0.130144
4,p118,0.133981


In [11]:
eng_vsd = eng_vsd.merge(engdur, on=["Participant"])
eng_vsd.head()

,is_stress,Area,Cutoff,Participant,Dom,avg_dur
0,1,4.017514,0.10,p111,L2,0.121433
1,0,3.348619,0.10,p111,L2,0.121433
2,1,3.508462,0.15,p111,L2,0.121433
3,0,2.990106,0.15,p111,L2,0.121433
4,1,3.429103,0.20,p111,L2,0.121433


In [12]:
spa_vsd.to_csv("data/spa_areas.csv", index = False)

In [13]:
eng_vsd.to_csv("data/eng_areas.csv", index = False)

In [14]:
cbas_eng = eng_vsd.copy()
cbas_eng["Language"] = "English"
cbas_spa = spa_vsd[spa_vsd["Dom"]!="mono"].copy()
cbas_spa["Language"] = "Spanish"
cbas_vsd = pd.concat([cbas_eng, cbas_spa])
cbas_vsd.sample(20)

,is_stress,Area,Cutoff,Participant,Dom,avg_dur,Language
53,0,3.301232,0.15,p119,biling,0.141896,Spanish
49,0,2.848389,0.30,p118,biling,0.113432,Spanish
8,1,2.832535,0.30,p111,L2,0.121433,English
76,1,3.646372,0.25,p122,biling,0.139943,English
12,1,4.267161,0.15,p113,biling,0.148272,English
18,1,3.68368,0.30,p113,biling,0.148272,English
19,0,2.000076,0.30,p113,biling,0.136585,Spanish
16,1,2.630129,0.25,p113,biling,0.136585,Spanish
107,0,2.127021,0.25,p126,L2,0.143693,English
78,1,2.242783,0.30,p122,biling,0.139943,English


In [15]:
cbas_vsd.to_csv("data/cbas_areas.csv", index = False)

In [2]:
import pandas as pd
spa = pd.read_csv("data/spa_areas.csv")
spa.head()

,is_stress,Area,Cutoff,Participant,Dom,avg_dur
0,1,3.302681,0.10,p111,L2,0.12047
1,0,3.539401,0.10,p111,L2,0.12047
2,1,3.032504,0.15,p111,L2,0.12047
3,0,3.023459,0.15,p111,L2,0.12047
4,1,2.816748,0.20,p111,L2,0.12047


In [5]:
spa["Corpus"] = spa["Dom"]!="mono"
spa.sample(5)

,is_stress,Area,Cutoff,Participant,Dom,avg_dur,Corpus
44,1,2.929383,0.20,p118,biling,0.113432,True
104,1,1.923681,0.20,p124,biling,0.112746,True
58,1,2.343957,0.30,p119,biling,0.141896,True
166,1,2.178490,0.25,s055,mono,0.081750,False
80,1,4.255101,0.10,p122,biling,0.122210,True


In [6]:
spa.to_csv("data/spa_cbas.csv", index = False)

In [42]:
cbas = pd.read_csv("data/cbas_areas.csv")
cbas.head()

,is_stress,Area,Cutoff,Participant,Dom,avg_dur,Language
0,1,4.017514,0.10,p111,L2,0.121433,English
1,0,3.348619,0.10,p111,L2,0.121433,English
2,1,3.508462,0.15,p111,L2,0.121433,English
3,0,2.990106,0.15,p111,L2,0.121433,English
4,1,3.429103,0.20,p111,L2,0.121433,English


In [44]:
cbas_list = pd.DataFrame(cbas.groupby(["Participant", "Language","Cutoff", "Dom", "avg_dur"])["Area"].agg(list))
cbas_list["Area_ratio"] = cbas_list["Area"].apply(lambda x: x[1]/x[0])
cbas_list.head()

Area  \
Participant Language Cutoff Dom avg_dur                                             
p111        English  0.10   L2  0.121433  [4.017514030303099, 3.3486187218481938]   
                     0.15   L2  0.121433  [3.5084623046006245, 2.990105514519839]   
                     0.20   L2  0.121433  [3.4291032144182463, 2.464699785629263]   
                     0.25   L2  0.121433   [3.076346920839076, 2.178923553410677]   
                     0.30   L2  0.121433  [2.8325346761905905, 2.108241617979124]   

                                          Area_ratio  
Participant Language Cutoff Dom avg_dur               
p111        English  0.10   L2  0.121433    0.833505  
                     0.15   L2  0.121433    0.852255  
                     0.20   L2  0.121433    0.718759  
                     0.25   L2  0.121433    0.708283  
                     0.30   L2  0.121433    0.744295

In [45]:
cbas_list = cbas_list.reset_index(drop = False)
cbas_list.head()

,Participant,Language,Cutoff,Dom,avg_dur,Area,Area_ratio
0,p111,English,0.10,L2,0.121433,"[4.017514030303099, 3.3486187218481938]",0.833505
1,p111,English,0.15,L2,0.121433,"[3.5084623046006245, 2.990105514519839]",0.852255
2,p111,English,0.20,L2,0.121433,"[3.4291032144182463, 2.464699785629263]",0.718759
3,p111,English,0.25,L2,0.121433,"[3.076346920839076, 2.178923553410677]",0.708283
4,p111,English,0.30,L2,0.121433,"[2.8325346761905905, 2.108241617979124]",0.744295


In [46]:
cbas_list = cbas_list.drop(["Area"], axis = 1)
cbas_list.head()

,Participant,Language,Cutoff,Dom,avg_dur,Area_ratio
0,p111,English,0.10,L2,0.121433,0.833505
1,p111,English,0.15,L2,0.121433,0.852255
2,p111,English,0.20,L2,0.121433,0.718759
3,p111,English,0.25,L2,0.121433,0.708283
4,p111,English,0.30,L2,0.121433,0.744295


In [49]:
cbas_list.sample(60)

,Participant,Language,Cutoff,Dom,avg_dur,Area_ratio
43,p118,English,0.25,biling,0.133981,0.627466
86,p122,Spanish,0.15,biling,0.122210,0.900878
40,p118,English,0.10,biling,0.133981,0.942999
36,p117,Spanish,0.15,L2,0.110605,1.100988
48,p118,Spanish,0.25,biling,0.113432,1.056383
2,p111,English,0.20,L2,0.121433,0.718759
0,p111,English,0.10,L2,0.121433,0.833505
84,p122,English,0.30,biling,0.139943,0.988295
34,p117,English,0.30,L2,0.130144,1.165189
35,p117,Spanish,0.10,L2,0.110605,1.201820


In [47]:
cbas_list.to_csv("data/cbas_ratios.csv", index = False)

In [39]:
spa = pd.read_csv("data/spa_areas.csv")
spa.head(50)

,is_stress,Area,Cutoff,Participant,Dom,avg_dur
0,1,3.302681,0.10,p111,L2,0.120470
1,0,3.539401,0.10,p111,L2,0.120470
2,1,3.032504,0.15,p111,L2,0.120470
3,0,3.023459,0.15,p111,L2,0.120470
4,1,2.816748,0.20,p111,L2,0.120470
5,0,2.852190,0.20,p111,L2,0.120470
6,1,2.552443,0.25,p111,L2,0.120470
7,0,2.712853,0.25,p111,L2,0.120470
8,1,2.394428,0.30,p111,L2,0.120470
9,0,2.589947,0.30,p111,L2,0.120470


In [35]:
spa_list = pd.DataFrame(spa.groupby(["Participant", "Cutoff", "Dom", "avg_dur"])["Area"].agg(list))
spa_list["Area_ratio"] = spa_list["Area"].apply(lambda x: x[1]/x[0])
spa_list.head()

Area  \
Participant Cutoff Dom avg_dur                                             
p111        0.10   L2  0.12047  [3.3026809703024367, 3.5394013984844914]   
            0.15   L2  0.12047  [3.0325041482489112, 3.0234585075031206]   
            0.20   L2  0.12047  [2.8167481848160705, 2.8521900566246963]   
            0.25   L2  0.12047   [2.552443317536168, 2.7128532528829874]   
            0.30   L2  0.12047  [2.3944277317889227, 2.5899467535805867]   

                                Area_ratio  
Participant Cutoff Dom avg_dur              
p111        0.10   L2  0.12047    1.071675  
            0.15   L2  0.12047    0.997017  
            0.20   L2  0.12047    1.012583  
            0.25   L2  0.12047    1.062846  
            0.30   L2  0.12047    1.081656

In [36]:
spa_list = spa_list.drop(["Area"], axis = 1)
spa_list.head()

Area_ratio
Participant Cutoff Dom avg_dur            
p111        0.10   L2  0.12047    1.071675
            0.15   L2  0.12047    0.997017
            0.20   L2  0.12047    1.012583
            0.25   L2  0.12047    1.062846
            0.30   L2  0.12047    1.081656

In [37]:
spa_list = spa_list.reset_index(drop = False)
spa_list.head()

,Participant,Cutoff,Dom,avg_dur,Area_ratio
0,p111,0.10,L2,0.12047,1.071675
1,p111,0.15,L2,0.12047,0.997017
2,p111,0.20,L2,0.12047,1.012583
3,p111,0.25,L2,0.12047,1.062846
4,p111,0.30,L2,0.12047,1.081656


In [41]:
spa_list.tail(50)

,Participant,Cutoff,Dom,avg_dur,Area_ratio
40,p122,0.10,biling,0.122210,0.944772
41,p122,0.15,biling,0.122210,0.900878
42,p122,0.20,biling,0.122210,0.827414
43,p122,0.25,biling,0.122210,0.779356
44,p122,0.30,biling,0.122210,0.785435
45,p123,0.10,L2,0.131626,0.958471
46,p123,0.15,L2,0.131626,0.935336
47,p123,0.20,L2,0.131626,0.937153
48,p123,0.25,L2,0.131626,0.883921
49,p123,0.30,L2,0.131626,0.843680


In [38]:
spa_list.to_csv("data/spa_ratios.csv", index = False)